In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [ ]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 14.5 MB/s eta 0:00:00


In [ ]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from optimizers_impl.gsam import GSAM, CosineScheduler, ProportionScheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        def loss_fn(predictions, targets):
            return smooth_crossentropy(predictions, targets, smoothing=0.1).mean()

        optimizer.set_closure(loss_fn, inputs, targets)
        predictions, loss = optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler.step()
            optimizer.update_rho_t()


In [ ]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())


In [ ]:
# if __name__ == "__main__":

#     num_epochs = 50
#     model_name = '/content/drive/MyDrive/ColabNotebooks/results_logs/widerestnet-gsam'
#     # model_name = 'widerestnet-gsam'
#     initialize(seed=111)
#     log = Log(log_each=1, log_name=model_name)

#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

#     base_optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.6, momentum=0.55)
#     dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)

#     scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.01, min_value=0.0, optimizer=base_optimizer)
#     rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.001, min_lr=0.0, max_value=0.1, min_value=0.0)
#     optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.0001, rho_scheduler=rho_scheduler, adaptive=True)

#     for epoch in range(num_epochs):
#         train(model, device, dataset.train, optimizer, log, scheduler)
#         test(model, device, dataset.valid, log)

#     log.flush()


In [ ]:
if __name__ == "__main__":

    num_epochs = 50
    # model_name = '/content/drive/MyDrive/ColabNotebooks/results_logs/widerestnet-gsam'
    model_name = 'widerestnet-gsam'
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

    base_optimizer = torch.optim.SGD(model.parameters(), lr=0.005, weight_decay=0.0, momentum=0.0)
    dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)

    scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.01, min_value=0.0, optimizer=base_optimizer)
    rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.001, min_lr=0.0, max_value=0.1, min_value=0.0)
    optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.5, rho_scheduler=rho_scheduler, adaptive=True)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      0.3634  │     56.11 %  ┃   9.994e-03  │       00:00  ┃      0.3671  │     41.33 %  ┃
┃           1  ┃      0.3512  │     62.89 %  ┃   9.969e-03  │       00:00  ┃      0.3641  │     71.33 %  ┃
┃           2  ┃      0.3407  │     78.00 %  ┃   9.924e-03  │       00:00  ┃      0.3616  │     57.00 %  ┃
┃           3  ┃      0.3301  │     74.11 %  ┃   9.860e-03  │       00:00  ┃      0.3580  │     67.67 %  ┃
┃           4  ┃      0.3206  │     78.11 %  ┃   9.776e-03  │       00:00  ┃      0.3557  │     70.33 %  ┃
┃           5  ┃      0.3073  │     7

In [ ]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [ ]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)



tensor(0.9524, device='cuda:0')
tensor(0.9500, device='cuda:0')
tensor(0.9091, device='cuda:0')
tensor(0.9000, device='cuda:0')
tensor(1., device='cuda:0')
